ekstraktowanie cech związanych z pokryciem terenu

In [25]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon, MultiPoint
from scipy.spatial.distance import cdist
import numpy as np

In [26]:
# pobranie granic miejscowości już po usunięciu tych bez własnej adrsacji
granice_miejscowosci_z_adresacja = gpd.read_file('28/warminskie_granice_miejscowosci_z_adresacja.shp')
granice_miejscowosci_z_adresacja

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,EM_LOKID,EM_PNAZW,EM_WERID,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,area,layer,path,geometry
0,2801,2C98CCB7-E677-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0177,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474382,PL.ZIPIN.870,NaN,109502,PL.PZGiK.204.PRNG,NaN,2773713.13,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602428.650 708821.710, 602501.620 70..."
1,2801,2C98CCB7-E663-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0171,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474324,PL.ZIPIN.870,NaN,99197,PL.PZGiK.204.PRNG,NaN,6744120.39,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((600581.150 708206.820, 600756.320 70..."
2,2801,2C98CCB7-E68D-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.0196,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474442,PL.ZIPIN.870,NaN,126609,PL.PZGiK.204.PRNG,NaN,7996737.92,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((597062.200 717054.300, 597160.520 71..."
3,2801,2C98CCB7-E690-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01108,ADMS03,NaN,inapplicable,dokladny,NaN,...,0474465,PL.ZIPIN.870,NaN,140116,PL.PZGiK.204.PRNG,NaN,3882361.42,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602589.080 723526.660, 602670.420 72..."
4,2801,2C98CCB7-E66F-AAFE-E053-CA2BA8C0B325,2022-03-01T12:46:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.ADMSA.28.01228,ADMS07,NaN,inapplicable,przyblizony,NaN,...,0474420,PL.ZIPIN.870,NaN,127660,PL.PZGiK.204.PRNG,NaN,129325.10,PL.PZGiK.341.2801__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((599604.680 721448.040, 599629.000 72..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3768,2819,311304D1-499A-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.104,ADMS03,NaN,inapplicable,dokladny,1.0,...,765116,PL.ZIPIN.1943,NaN,00000000-0000-0000-0000-000000040858,PL.PZGiK.204.PRNG,NaN,47631769.37,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((682986.450 703211.410, 682991.700 70..."
3769,2819,311304D1-4996-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.103,ADMS03,NaN,inapplicable,dokladny,1.0,...,770979,PL.ZIPIN.903,NaN,00000000-0000-0000-0000-000000040171,PL.PZGiK.204.PRNG,NaN,22742342.71,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((667907.680 715869.080, 667852.960 71..."
3770,2819,311304D1-49C6-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.116,ADMS07,NaN,inapplicable,dokladny,1.0,...,771341,PL.ZIPIN.903,NaN,00000000-0000-0000-0000-000000133740,PL.PZGiK.204.PRNG,NaN,3079021.72,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((669232.280 706589.380, 669260.150 70..."
3771,2819,311304D1-49D7-B2CD-E053-CC2BA8C0FE3A,2021-10-06T13:36:00,PL.PZGiK.341.BDOT10k,OT_ADMS_A.47,ADMS07,NaN,inapplicable,dokladny,1.0,...,755632,PL.ZIPIN.902,NaN,00000000-0000-0000-0000-000000026309,PL.PZGiK.204.PRNG,NaN,2833038.18,PL.PZGiK.341.2819__OT_ADMS_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((682807.210 712243.900, 682802.600 71..."


In [28]:
# ograniczenie warstwy granaic tylko do dwóch kolumn
granice_temp = granice_miejscowosci_z_adresacja[['SIMC_id', 'geometry']]
granice_temp

,SIMC_id,geometry
0,474382,"POLYGON ((602428.650 708821.710, 602501.620 70..."
1,474324,"POLYGON ((600581.150 708206.820, 600756.320 70..."
2,474442,"POLYGON ((597062.200 717054.300, 597160.520 71..."
3,474465,"POLYGON ((602589.080 723526.660, 602670.420 72..."
4,474420,"POLYGON ((599604.680 721448.040, 599629.000 72..."
...,...,...
3768,765116,"POLYGON ((682986.450 703211.410, 682991.700 70..."
3769,770979,"POLYGON ((667907.680 715869.080, 667852.960 71..."
3770,771341,"POLYGON ((669232.280 706589.380, 669260.150 70..."
3771,755632,"POLYGON ((682807.210 712243.900, 682802.600 71..."


teren leśny i zadrzewiony

In [27]:
# pobranie warstwy z terenami zalesionymi
lasy = gpd.read_file('28/warminskie_lasy.shp')
lasy

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,BD500_ID,BD500_PNA,BD500_WEID,area,layer,path,geometry
0,2801,D746AA4C-80A1-41E1-8075-81CFF3195E8E,2020-10-16T00:00:00,PL.PZGiK.341.BDOT10k,ID_D746AA4C-80A1-41E1-8075-81CFF3195E8E,PTLZ02,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2865.86,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((612972.120 722130.510, 612897.120 72..."
1,2801,F21572D2-1C03-450A-B14F-CF29536901A1,2020-10-08T00:00:00,PL.PZGiK.341.BDOT10k,ID_F21572D2-1C03-450A-B14F-CF29536901A1,PTLZ02,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22323.00,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((594321.600 722200.330, 594252.270 72..."
2,2801,F670BB7C-6CB2-4B04-A291-22AB551E650F,2020-10-09T00:00:00,PL.PZGiK.341.BDOT10k,ID_F670BB7C-6CB2-4B04-A291-22AB551E650F,PTLZ02,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1530.55,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((596860.440 723806.370, 596827.840 72..."
3,2801,DA8E0C62-2C5D-4DB6-8727-37B5BBE7337A,2020-10-10T00:00:00,PL.PZGiK.341.BDOT10k,ID_DA8E0C62-2C5D-4DB6-8727-37B5BBE7337A,PTLZ01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,39377.50,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((598952.360 724876.780, 598928.110 72..."
4,2801,B5068A0E-E9AD-4680-BBDC-02C19705AA8D,2020-10-12T00:00:00,PL.PZGiK.341.BDOT10k,ID_B5068A0E-E9AD-4680-BBDC-02C19705AA8D,PTLZ02,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36082.73,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((603539.930 716636.310, 603499.870 71..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122663,2862,2BF83540-FE93-83A2-E053-CC2BA8C04E93,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTLZA.28.62327,PTLZ01,NaN,inapplicable,przyblizony,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1755.87,PL.PZGiK.341.2862__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((597228.240 654230.800, 597224.210 65..."
122664,2862,ced8be69-2293-11eb-b58f-480fcf3c9bd7,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A_1604930378_51,PTLZ01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2029.20,PL.PZGiK.341.2862__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((598152.950 656868.690, 598159.290 65..."
122665,2862,ced64d67-2293-11eb-82fd-480fcf3c9bd7,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A_1604930378_28,PTLZ02,NaN,inapplicable,przyblizony,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4944.27,PL.PZGiK.341.2862__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((600077.190 660514.020, 599986.150 66..."
122666,2862,ced64d6d-2293-11eb-849a-480fcf3c9bd7,2022-08-08T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A_1604930378_34,PTLZ01,NaN,inapplicable,przyblizony,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22328.18,PL.PZGiK.341.2862__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((599727.950 655064.390, 599730.680 65..."


In [29]:
# intersekcja warstwy z granicami miejscowości - podzielenie warstwy wg granic miejscowości i dopisanie simc_id
lasy_grouped = gpd.overlay(granice_temp, lasy, how='intersection')
lasy_grouped

C:\Users\agnie\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
C:\Users\agnie\AppData\Local\Temp\ipykernel_19248\3240215390.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 21885 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  lasy_grouped = gpd.overlay(granice_temp, lasy, how='intersection')


,SIMC_id,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,...,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,BD500_ID,BD500_PNA,BD500_WEID,area,layer,path,geometry
0,474382,2801,2C98CCB8-DB22-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTLZA.28.011638,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,3920.81,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602310.060 706727.720, 602340.300 70..."
1,474324,2801,2C98CCB8-DB22-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTLZA.28.011638,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,3920.81,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602340.300 706847.240, 602310.060 70..."
2,474382,2801,2C98CCB8-DF54-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTLZA.28.012528,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,8596.00,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602882.160 707716.370, 602878.300 70..."
3,474382,2801,2C98CCB8-D8AC-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTLZA.28.011094,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,50408.59,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((603272.650 707226.580, 603207.614 70..."
4,471320,2801,2C98CCB8-D8AC-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTLZA.28.011094,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,50408.59,PL.PZGiK.341.2801__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((603272.650 707226.580, 603285.454 70..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151810,755655,2819,311304D1-E69A-B2CD-E053-CC2BA8C0FE3A,2019-03-06T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A.1843,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,1994.62,PL.PZGiK.341.2819__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690540.530 722577.220, 690551.580 72..."
151811,755655,2819,311304D1-E683-B2CD-E053-CC2BA8C0FE3A,2019-03-06T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A.912,PTLZ03,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,2218.21,PL.PZGiK.341.2819__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690416.530 721532.520, 690393.940 72..."
151812,755655,2819,311304D1-E867-B2CD-E053-CC2BA8C0FE3A,2019-03-06T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A.908,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,1660.03,PL.PZGiK.341.2819__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690601.520 722743.180, 690611.440 72..."
151813,755655,2819,311304D1-E366-B2CD-E053-CC2BA8C0FE3A,2019-03-06T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTLZ_A.909,PTLZ01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,1329.08,PL.PZGiK.341.2819__OT_PTLZ_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690597.880 722578.770, 690599.530 72..."


In [30]:
# stworzenie multipoligonów odpowiadającym terenom zalesionym konkretnych numerów miejscowości
lasy_merged = lasy_grouped[['SIMC_id', 'geometry']].dissolve(by='SIMC_id').reset_index()
lasy_merged

,SIMC_id,geometry
0,113419,"MULTIPOLYGON (((576457.390 598801.480, 576383...."
1,113431,"MULTIPOLYGON (((579842.572 589988.063, 579846...."
2,113448,"MULTIPOLYGON (((576047.860 602181.660, 576060...."
3,113454,"MULTIPOLYGON (((577797.510 614247.010, 577777...."
4,113460,"MULTIPOLYGON (((572350.840 591649.530, 572270...."
...,...,...
3687,1063126,"MULTIPOLYGON (((567906.525 681834.501, 567904...."
3688,1064491,"POLYGON ((557888.550 719800.930, 557887.518 71..."
3689,1064500,"POLYGON ((546381.310 721688.060, 546405.170 72..."
3690,1064516,"MULTIPOLYGON (((660037.640 640333.320, 659961...."


In [24]:
# zapis do pliku takiej warstwy pomocniczej
# lasy_merged.to_file('28/test2_lasy_grouped.shp')

In [49]:
# łączna powierzchnia zabudowy
powierzchnia_terenow_zalesionych = lasy_grouped.groupby('SIMC_id')['area'].sum().reset_index()
powierzchnia_terenow_zalesionych.rename(columns={'area': 'powierzchnia_terenow_zalesionych'}, inplace=True)
powierzchnia_terenow_zalesionych

,SIMC_id,powierzchnia_terenow_zalesionych
0,113419,10056907.35
1,113431,3183391.75
2,113448,11459110.83
3,113454,184155.46
4,113460,1444156.94
...,...,...
3687,1063126,539170.79
3688,1064491,247222.36
3689,1064500,211672.36
3690,1064516,61691953.63


roślinność trawiasta i uprawa rolna

In [33]:
# pobranie warstwy z terenami uprawnymi
pola = gpd.read_file('28/warminskie_uprawa.shp')
pola

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,EGIB_LOKID,EGIB_PNAZW,EGIB_WERID,area,layer,path,geometry
0,2801,2C98CCB8-F5CE-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.012652,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5632.87,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((620280.750 713906.170, 620269.310 71..."
1,2801,2C98CCB8-F8DD-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.014362,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2730.99,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((632892.190 713862.830, 632840.960 71..."
2,2801,2C98CCB8-F6A0-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.015770,PTTR02,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,33220.28,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((628129.670 710079.050, 628163.690 71..."
3,2801,2C98CCB8-F6A2-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.015772,PTTR02,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14825.83,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((628489.500 710393.700, 628448.610 71..."
4,2801,2C98CCB8-FA1D-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.014716,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2848.93,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((642337.660 717922.910, 642336.910 71..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91085,2862,LwwWNWjN-HWoj-jJUw-o29U-ijiYzIFKrInO,2023-04-11T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.70604542-8ac2-4931-af37-4b2447e4bebe,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,29551.44,PL.PZGiK.341.2862__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((594244.460 655523.520, 594234.190 65..."
91086,2862,yxTBqpiE-K7e1-q3nR-F5TR-0GYT8SpRPBFi,2023-04-11T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.d91fce62-ee55-480d-a7de-4d056970e826,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,656630.00,PL.PZGiK.341.2862__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((593898.370 655626.930, 593861.050 65..."
91087,2862,DW7v7dUH-Qv7u-CCJm-GRDs-fflcHGbH2b0r,2023-04-11T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.e71f0ee6-24bf-4b32-b639-cf84a15abbfb,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8496.65,PL.PZGiK.341.2862__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((594283.160 656336.510, 594278.280 65..."
91088,2862,gdXyGOcX-FaeL-fGH4-9gUp-uO28Jn1ESDv4,2023-04-11T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.e19c11c3-b908-48a4-9b07-725f09c9b632,PTTR01,NaN,inapplicable,dokladny,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,143814.69,PL.PZGiK.341.2862__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"POLYGON ((599598.330 657842.250, 599603.940 65..."


In [34]:
# intersekcja warstwy z granicami miejscowości - podzielenie warstwy wg granic miejscowości i dopisanie simc_id
pola_grouped = gpd.overlay(granice_temp, pola, how='intersection')
pola_grouped

C:\Users\agnie\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
C:\Users\agnie\AppData\Local\Temp\ipykernel_19248\1673816056.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 18957 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  pola_grouped = gpd.overlay(granice_temp, pola, how='intersection')


,SIMC_id,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,...,PRNG_LOKID,PRNG_PNAZW,PRNG_WERID,EGIB_LOKID,EGIB_PNAZW,EGIB_WERID,area,layer,path,geometry
0,474382,2801,2C98CCB8-ED95-AAFE-E053-CA2BA8C0B325,2021-09-09T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.011128,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,831949.13,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"MULTIPOLYGON (((602610.450 708747.620, 602608...."
1,474525,2801,2C98CCB8-ED95-AAFE-E053-CA2BA8C0B325,2021-09-09T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.011128,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,831949.13,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"MULTIPOLYGON (((602642.540 709733.410, 602642...."
2,471171,2801,2C98CCB8-ED95-AAFE-E053-CA2BA8C0B325,2021-09-09T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.011128,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,831949.13,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((603021.150 708668.530, 602909.020 70..."
3,473945,2801,2C98CCB8-ED95-AAFE-E053-CA2BA8C0B325,2021-09-09T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.PTTRA.28.011128,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,831949.13,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"MULTIPOLYGON (((602763.161 709192.423, 602762...."
4,474382,2801,96bd3f73-3c61-11e9-a797-f01faf546c5e,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A_1551472543_62,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,5182.94,PL.PZGiK.341.2801__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"POLYGON ((602598.800 707809.640, 602595.640 70..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113660,755655,2819,311304D1-F554-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.636,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,1591.49,PL.PZGiK.341.2819__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690343.240 721512.200, 690329.570 72..."
113661,755655,2819,311304D1-F555-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.637,PTTR02,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,2484.57,PL.PZGiK.341.2819__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690337.600 721579.020, 690355.020 72..."
113662,755655,2819,311304D1-F14E-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.583,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,63340.46,PL.PZGiK.341.2819__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((691148.140 720373.580, 691137.370 72..."
113663,755655,2819,311304D1-F293-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_PTTR_A.627,PTTR01,NaN,inapplicable,dokladny,...,NaN,NaN,NaN,NaN,NaN,NaN,3315.53,PL.PZGiK.341.2819__OT_PTTR_A,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,"POLYGON ((690381.700 721667.040, 690414.670 72..."


In [36]:
# stworzenie multipoligonów odpowiadającym terenom uprawnym konkretnych numerów miejscowości
pola_merged = pola_grouped[['SIMC_id', 'geometry']].dissolve(by='SIMC_id').reset_index()
pola_merged

,SIMC_id,geometry
0,113419,"MULTIPOLYGON (((576371.205 598732.551, 576383...."
1,113431,"MULTIPOLYGON (((578889.730 590409.470, 578823...."
2,113448,"MULTIPOLYGON (((576365.500 602161.670, 576344...."
3,113454,"MULTIPOLYGON (((577141.526 614484.133, 577062...."
4,113460,"MULTIPOLYGON (((572095.550 591904.800, 572095...."
...,...,...
3748,1063126,"POLYGON ((567808.020 682413.650, 567808.610 68..."
3749,1064491,"MULTIPOLYGON (((557921.640 719490.270, 557901...."
3750,1064500,"POLYGON ((546237.920 721516.280, 546247.940 72..."
3751,1064516,"MULTIPOLYGON (((660159.740 640282.970, 660140...."


In [37]:
# zapis do pliku takiej warstwy pomocniczej
pola_merged.to_file('28/uprawa_grouped.shp')

In [48]:
# łączna powierzchnia pól uprawnych
powierzchnia_terenow_uprawnych = pola_grouped.groupby('SIMC_id')['area'].sum().reset_index()
powierzchnia_terenow_uprawnych.rename(columns={'area': 'powierzchnia_terenow_uprawnych'}, inplace=True)
powierzchnia_terenow_uprawnych

,SIMC_id,powierzchnia_terenow_uprawnych
0,113419,44778913.48
1,113431,36087179.85
2,113448,40700039.03
3,113454,63139137.64
4,113460,41841306.61
...,...,...
3748,1063126,16773940.84
3749,1064491,10569385.85
3750,1064500,2019353.15
3751,1064516,1568175.96


drogi

In [39]:
# pobranie warstwy z drogami
drogi = gpd.read_file('28/warminskie_drogi.shp')
drogi

,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,NAZWADROGI,LJEZDNIDR,LJEZD_NIL,NRODCIREF,SZL_NUMER,SZLNR_NIL,length,layer,path,geometry
0,2801,86C1C68E-5E18-4250-86E1-204E6182D9BF,2020-11-13T00:00:00,PL.PZGiK.341.BDOT10k,ID_86C1C68E-5E18-4250-86E1-204E6182D9BF,SKDR06,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,119027N,NaN,233.85,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"LINESTRING (602143.350 712305.560, 602150.010 ..."
1,2801,DD5FE975-3BE7-483B-8131-0EE9D9C15D18,2020-10-20T00:00:00,PL.PZGiK.341.BDOT10k,ID_DD5FE975-3BE7-483B-8131-0EE9D9C15D18,SKDR04,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,511|512,NaN,291.55,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"LINESTRING (597575.030 714125.740, 597553.710 ..."
2,2801,6D51BDEF-B869-4BB4-975A-5116F5F21A63,2020-10-22T00:00:00,PL.PZGiK.341.BDOT10k,ID_6D51BDEF-B869-4BB4-975A-5116F5F21A63,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,23.46,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"LINESTRING (594691.720 718366.060, 594687.660 ..."
3,2801,8068F876-8E91-44B8-9C0B-52BFBBBCF63A,2020-10-20T00:00:00,PL.PZGiK.341.BDOT10k,ID_8068F876-8E91-44B8-9C0B-52BFBBBCF63A,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,20.94,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"LINESTRING (618860.920 711296.700, 618881.400 ..."
4,2801,8299B2BC-A9A0-46AE-AF9C-1309D47BB741,2020-10-20T00:00:00,PL.PZGiK.341.BDOT10k,ID_8299B2BC-A9A0-46AE-AF9C-1309D47BB741,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,62.81,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,"LINESTRING (619332.450 713000.980, 619327.340 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297103,2862,2BF83541-12A8-83A2-E053-CC2BA8C04E93,2023-04-05T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.SKDRL.28.621483,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,115.57,PL.PZGiK.341.2862__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"LINESTRING (591144.120 660676.230, 591151.110 ..."
297104,2862,2BF83541-1093-83A2-E053-CC2BA8C04E93,2023-04-11T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.SKDRL.28.621514,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,253.70,PL.PZGiK.341.2862__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"LINESTRING (592123.000 657722.760, 592123.530 ..."
297105,2862,d9647b35-a8da-11eb-a73b-b8ca3aa2e5c2,2023-04-06T00:00:00,PL.PZGiK.341.BDOT10k,OT_SKDR_L_1619694344_119,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,137.14,PL.PZGiK.341.2862__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"LINESTRING (600041.420 661846.580, 600047.580 ..."
297106,2862,4a117d5e-3bbd-11eb-a7fc-5404a6a68038,2023-04-05T00:00:00,PL.PZGiK.341.BDOT10k,OT_SKDR_L_1607696972_25,SKDR08,NaN,inapplicable,dokladny,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,134.63,PL.PZGiK.341.2862__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2862_SHP\PL.PZGiK.3...,"LINESTRING (591281.660 660521.830, 591274.040 ..."


In [41]:
# intersekcja warstwy z granicami miejscowości - podzielenie warstwy wg granic miejscowości i dopisanie simc_id
drogi_grouped = gpd.overlay(drogi, granice_temp, how='intersection')
drogi_grouped

C:\Users\agnie\AppData\Local\Temp\ipykernel_19248\3782198061.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 9644 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  drogi_grouped = gpd.overlay(drogi, granice_temp, how='intersection')


,TERYT,LOKALNYID,WERSJAID,PNAZW,GMLID,X_KOD,X_SKRKARTO,X_SKRK_NIL,X_KATDOKG,X_DOKLGEOM,...,LJEZDNIDR,LJEZD_NIL,NRODCIREF,SZL_NUMER,SZLNR_NIL,length,layer,path,SIMC_id,geometry
0,2801,86C1C68E-5E18-4250-86E1-204E6182D9BF,2020-11-13T00:00:00,PL.PZGiK.341.BDOT10k,ID_86C1C68E-5E18-4250-86E1-204E6182D9BF,SKDR06,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,119027N,NaN,233.85,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,473974,"LINESTRING (602143.350 712305.560, 602150.010 ..."
1,2801,E490DBAD-7958-4D54-BC5E-AB324CAD66C1,2020-11-13T00:00:00,PL.PZGiK.341.BDOT10k,ID_E490DBAD-7958-4D54-BC5E-AB324CAD66C1,SKDR06,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,119028N,NaN,383.98,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,473974,"LINESTRING (602922.430 712105.970, 602956.270 ..."
2,2801,76F4E04B-1628-4EB3-804E-54A3245E28FE,2020-11-13T00:00:00,PL.PZGiK.341.BDOT10k,ID_76F4E04B-1628-4EB3-804E-54A3245E28FE,SKDR06,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,119028N,NaN,67.60,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,473974,"LINESTRING (602232.570 712098.310, 602255.580 ..."
3,2801,2C98CCB9-2886-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.SKDRL.28.011152,SKDR06,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,119027N,NaN,23.87,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,473974,"LINESTRING (602132.780 712326.900, 602138.030 ..."
4,2801,2C98CCB9-281E-AAFE-E053-CA2BA8C0B325,2020-09-25T00:00:00,PL.PZGiK.341.BDOT10k,PL.PZGIK.BDOT10k.SKDRL.28.011046,SKDR06,NaN,inapplicable,dokladny,NaN,...,1.0,NaN,NaN,119028N,NaN,639.61,PL.PZGiK.341.2801__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2801_SHP\PL.PZGiK.3...,473974,"LINESTRING (602298.020 712098.150, 602327.280 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287315,2819,311304D2-1E2A-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_SKDR_L.2318,SKDR06,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,1758N,NaN,420.51,PL.PZGiK.341.2819__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,755610,"LINESTRING (680855.440 716358.770, 680892.540 ..."
287316,2819,40af063c-07db-4f68-bc38-8b3c06ab7987,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_SKDR_L.5705,SKDR08,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,NaN,NaN,695.81,PL.PZGiK.341.2819__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,755477,"MULTILINESTRING ((679584.950 721560.940, 67954..."
287317,2819,311304D2-1DE6-B2CD-E053-CC2BA8C0FE3A,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_SKDR_L.2357,SKDR08,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,NaN,NaN,298.37,PL.PZGiK.341.2819__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,755477,"MULTILINESTRING ((679103.540 721097.800, 67910..."
287318,2819,83a10eed-a00e-4ad4-b30f-c38dab60fe00,2020-09-07T12:00:00,PL.PZGiK.341.BDOT10k,OT_SKDR_L.5536,SKDR08,NaN,inapplicable,dokladny,1.0,...,1.0,NaN,NaN,NaN,NaN,996.99,PL.PZGiK.341.2819__OT_SKDR_L,C:\studia\pracka\28_SHP\28\2819_SHP\PL.PZGiK.3...,755649,"LINESTRING (683832.595 711592.135, 683841.870 ..."


In [43]:
# stworzenie multilineów odpowiadającym drogom konkretnym numerów miejscowości
drogi_merged = drogi_grouped[['SIMC_id', 'geometry']].dissolve(by='SIMC_id').reset_index()
drogi_merged

,SIMC_id,geometry
0,113419,"MULTILINESTRING ((576953.710 600084.860, 57694..."
1,113431,"MULTILINESTRING ((578535.709 591468.860, 57853..."
2,113448,"MULTILINESTRING ((575941.750 603159.740, 57593..."
3,113454,"MULTILINESTRING ((577808.620 615018.120, 57782..."
4,113460,"MULTILINESTRING ((573020.190 592584.950, 57302..."
...,...,...
3757,1063126,"MULTILINESTRING ((568091.700 682143.290, 56809..."
3758,1064491,"MULTILINESTRING ((557868.430 720007.950, 55782..."
3759,1064500,"MULTILINESTRING ((546475.667 721619.021, 54647..."
3760,1064516,"MULTILINESTRING ((660908.530 644071.891, 66090..."


In [44]:
# zapis do pliku takiej warstwy pomocniczej
drogi_merged.to_file('28/drogi_grouped.shp')

In [51]:
# łączna długość dróg
dlugosc_drog = drogi_grouped.groupby('SIMC_id')['length'].sum().reset_index()
dlugosc_drog.rename(columns={'length':'dlugosc_drog'}, inplace=True)
dlugosc_drog

,SIMC_id,dlugosc_drog
0,113419,41187.50
1,113431,12049.35
2,113448,61774.28
3,113454,12008.31
4,113460,9195.02
...,...,...
3757,1063126,4766.21
3758,1064491,6032.00
3759,1064500,808.30
3760,1064516,22908.80


połączenie wszystkich wyekstraktowanych cech

In [52]:
cechy_merged = dlugosc_drog.merge(powierzchnia_terenow_uprawnych, how='left', on='SIMC_id')
cechy_merged.drop_duplicates(inplace=True)
cechy_merged

,SIMC_id,dlugosc_drog,powierzchnia_terenow_uprawnych
0,113419,41187.50,44778913.48
1,113431,12049.35,36087179.85
2,113448,61774.28,40700039.03
3,113454,12008.31,63139137.64
4,113460,9195.02,41841306.61
...,...,...,...
3757,1063126,4766.21,16773940.84
3758,1064491,6032.00,10569385.85
3759,1064500,808.30,2019353.15
3760,1064516,22908.80,1568175.96


In [53]:
cechy_merged = cechy_merged.merge(powierzchnia_terenow_zalesionych, how='left', on='SIMC_id')
cechy_merged.drop_duplicates(inplace=True)
cechy_merged

,SIMC_id,dlugosc_drog,powierzchnia_terenow_uprawnych,powierzchnia_terenow_zalesionych
0,113419,41187.50,44778913.48,10056907.35
1,113431,12049.35,36087179.85,3183391.75
2,113448,61774.28,40700039.03,11459110.83
3,113454,12008.31,63139137.64,184155.46
4,113460,9195.02,41841306.61,1444156.94
...,...,...,...,...
3757,1063126,4766.21,16773940.84,539170.79
3758,1064491,6032.00,10569385.85,247222.36
3759,1064500,808.30,2019353.15,211672.36
3760,1064516,22908.80,1568175.96,61691953.63


In [54]:
# zbadanie, czy występują wartości null albo nan
na = cechy_merged[cechy_merged.isnull().any(axis=1)]
na

,SIMC_id,dlugosc_drog,powierzchnia_terenow_uprawnych,powierzchnia_terenow_zalesionych
24,113709,2095.82,2.183744e+06,NaN
37,116406,3825.08,3.179151e+07,NaN
38,116429,7695.75,6.517976e+07,NaN
57,117943,1278.53,NaN,6719021.90
84,118240,12437.37,5.438495e+07,NaN
...,...,...,...,...
3428,767150,647.29,NaN,47676763.95
3726,988738,574.94,NaN,3882078.40
3736,995939,3436.84,NaN,71458037.00
3738,995951,1039.39,2.748894e+05,NaN


In [55]:
cechy_merged.fillna(0, inplace=True)
cechy_merged

,SIMC_id,dlugosc_drog,powierzchnia_terenow_uprawnych,powierzchnia_terenow_zalesionych
0,113419,41187.50,44778913.48,10056907.35
1,113431,12049.35,36087179.85,3183391.75
2,113448,61774.28,40700039.03,11459110.83
3,113454,12008.31,63139137.64,184155.46
4,113460,9195.02,41841306.61,1444156.94
...,...,...,...,...
3757,1063126,4766.21,16773940.84,539170.79
3758,1064491,6032.00,10569385.85,247222.36
3759,1064500,808.30,2019353.15,211672.36
3760,1064516,22908.80,1568175.96,61691953.63


In [57]:
dane_warminskie = pd.read_csv('28/warminskie_miejscowosci_do_analizy.csv', sep=';')
dane_warminskie

,SIMC_id,rodzaj,nazwa,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,...,ogolnodostepneObiektyKulturalne,pozostaleBudynkiNiemieszkalne,zbiornikSilosIBudynkiMagazynowe,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep
0,149222,wieś,Adamowo,True,21,65,5184091.39,60.0,14715.06,1.300000,...,0.0,0.0,0.0,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024
1,157948,wieś,Adamowo,True,69,255,4841070.89,147.0,25168.40,1.408163,...,0.0,0.0,0.0,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003
2,117890,wieś,Adamowo,True,33,83,2073808.18,93.0,8551.89,1.505376,...,0.0,0.0,0.0,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759
3,471403,wieś,Adamowo,True,13,142,12131404.50,118.0,32306.85,1.101695,...,0.0,0.0,0.0,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913
4,149647,część wsi,Adamowo-Osiedle,False,26,88,264581.60,29.0,7311.39,1.448276,...,0.0,0.0,3.0,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1044100,osada,Rozdroże,True,76,429,944005.19,88.0,17862.33,1.409091,...,0.0,1.0,0.0,0.570609,397.641182,221.024272,55.583848,6.201421,1074.914468,1074.914468
3759,1047050,osada,Janowo,True,2,5,4442946.03,21.0,1600.72,1.285714,...,0.0,0.0,0.0,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443
3760,764660,kolonia,Wąglik-Kolonia,True,6,7,24333.51,7.0,883.66,1.000000,...,0.0,0.0,0.0,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274
3761,770672,kolonia,Zajdy,True,5,12,704732.13,20.0,2580.28,1.000000,...,0.0,0.0,0.0,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269


In [58]:
dane_warminskie_merged = dane_warminskie.merge(cechy_merged, how='left', on='SIMC_id')
dane_warminskie_merged.drop_duplicates(inplace=True)
dane_warminskie_merged

,SIMC_id,rodzaj,nazwa,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,...,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep,dlugosc_drog,powierzchnia_terenow_uprawnych,powierzchnia_terenow_zalesionych
0,149222,wieś,Adamowo,True,21,65,5184091.39,60.0,14715.06,1.300000,...,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024,15714.60,32101261.06,139241.58
1,157948,wieś,Adamowo,True,69,255,4841070.89,147.0,25168.40,1.408163,...,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003,22394.61,32340402.95,2060690.32
2,117890,wieś,Adamowo,True,33,83,2073808.18,93.0,8551.89,1.505376,...,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759,6936.66,13050833.55,926673.01
3,471403,wieś,Adamowo,True,13,142,12131404.50,118.0,32306.85,1.101695,...,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913,47810.99,16201419.47,9013457.10
4,149647,część wsi,Adamowo-Osiedle,False,26,88,264581.60,29.0,7311.39,1.448276,...,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649,5244.73,31493223.74,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1044100,osada,Rozdroże,True,76,429,944005.19,88.0,17862.33,1.409091,...,0.570609,397.641182,221.024272,55.583848,6.201421,1074.914468,1074.914468,5997.42,56509298.63,111774.63
3759,1047050,osada,Janowo,True,2,5,4442946.03,21.0,1600.72,1.285714,...,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443,20821.40,3788940.15,7144659.80
3760,764660,kolonia,Wąglik-Kolonia,True,6,7,24333.51,7.0,883.66,1.000000,...,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274,976.64,531362.10,12336688.10
3761,770672,kolonia,Zajdy,True,5,12,704732.13,20.0,2580.28,1.000000,...,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269,3008.37,6337753.61,913189.20


In [59]:
na = dane_warminskie_merged[dane_warminskie_merged.isnull().any(axis=1)]
na

,SIMC_id,rodzaj,nazwa,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,...,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep,dlugosc_drog,powierzchnia_terenow_uprawnych,powierzchnia_terenow_zalesionych
924,758671,osada leśna,Janki,True,1,4,7074.75,4.0,490.56,1.0,...,1.179456,21.847131,13.648670,62.473511,23.815491,41.857564,41.857564,NaN,NaN,NaN
3675,760604,osada leśna,Zawady Małe,True,1,2,4164.56,3.0,534.33,1.0,...,1.467033,20.040560,14.432551,72.016702,25.821662,34.015264,34.015264,NaN,NaN,NaN


In [60]:
dane_warminskie_merged.fillna(0, inplace=True)
dane_warminskie_merged

,SIMC_id,rodzaj,nazwa,czy_samodzielna,liczba_adresow,liczba_mieszkancow,powierzchnia,liczba_budynkow,powierzchnia_budynkow,srednia_liczba_kondygnacji,...,nna_index,srednia_odleglosci,odchylenie_standardowe,wspolczynnik_zmiennosci,min_odleglosc,max_odleglosc,min_max_rozstep,dlugosc_drog,powierzchnia_terenow_uprawnych,powierzchnia_terenow_zalesionych
0,149222,wieś,Adamowo,True,21,65,5184091.39,60.0,14715.06,1.300000,...,0.287704,1223.486507,853.187516,69.734117,6.805516,3049.927024,3049.927024,15714.60,32101261.06,139241.58
1,157948,wieś,Adamowo,True,69,255,4841070.89,147.0,25168.40,1.408163,...,0.345539,1211.098579,648.750812,53.567135,7.723816,3010.174003,3010.174003,22394.61,32340402.95,2060690.32
2,117890,wieś,Adamowo,True,33,83,2073808.18,93.0,8551.89,1.505376,...,0.232380,203.093999,156.541945,77.078568,5.140856,888.593759,888.593759,6936.66,13050833.55,926673.01
3,471403,wieś,Adamowo,True,13,142,12131404.50,118.0,32306.85,1.101695,...,0.160716,1239.796179,725.923757,58.551863,6.414879,3130.001913,3130.001913,47810.99,16201419.47,9013457.10
4,149647,część wsi,Adamowo-Osiedle,False,26,88,264581.60,29.0,7311.39,1.448276,...,0.454862,132.621878,79.351370,59.832790,5.648417,402.083649,402.083649,5244.73,31493223.74,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,1044100,osada,Rozdroże,True,76,429,944005.19,88.0,17862.33,1.409091,...,0.570609,397.641182,221.024272,55.583848,6.201421,1074.914468,1074.914468,5997.42,56509298.63,111774.63
3759,1047050,osada,Janowo,True,2,5,4442946.03,21.0,1600.72,1.285714,...,0.358676,1158.103589,1044.314695,90.174550,7.218277,2380.240443,2380.240443,20821.40,3788940.15,7144659.80
3760,764660,kolonia,Wąglik-Kolonia,True,6,7,24333.51,7.0,883.66,1.000000,...,1.081121,59.375132,34.049685,57.346710,24.023513,112.380274,112.380274,976.64,531362.10,12336688.10
3761,770672,kolonia,Zajdy,True,5,12,704732.13,20.0,2580.28,1.000000,...,0.165141,505.469867,361.880396,71.592872,7.672812,1110.875269,1110.875269,3008.37,6337753.61,913189.20


In [61]:
# zapis do csv
# dane_warminskie_merged.to_csv("28/warminskie_miejscowosci_do_analizy_v2.csv", sep=';', header=True, index=False)